<a href="https://colab.research.google.com/github/abasnezhad/120_years_of_Olympic/blob/main/LSTM_time_series_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!pip install -qU kaggle
!pip install -qU plotly

In [3]:
import numpy as np
import pandas as pd

import plotly
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing


import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import json
import datetime

%matplotlib inline

In [ ]:
#@title Kaggle Credential { display-mode: "form" }

username = '' #@param {type: "string"}
api_key = '' #@param {type: "string"}


if username and api_key:
    token = {"username": username, "key": api_key}

    !mkdir ~/.kaggle
    !mkdir /content/.kaggle
    with open('/content/.kaggle/kaggle.json', 'w') as f:
        json.dump(token, f)

    !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json

    print('Your are ready to use kaggle API!')

In [4]:
!mkdir /content/dataset
!kaggle datasets download borismarjanovic/price-volume-data-for-all-us-stocks-etfs
!mv price-volume-data-for-all-us-stocks-etfs.zip /content/dataset/price-volume-data-for-all-us-stocks-etfs.zip
!unzip /content/dataset/price-volume-data-for-all-us-stocks-etfs.zip -d /content/dataset/

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Stocks/ergf.us.txt  
  inflating: /content/dataset/Stocks/erh.us.txt  
  inflating: /content/dataset/Stocks/eri.us.txt  
  inflating: /content/dataset/Stocks/eric.us.txt  
  inflating: /content/dataset/Stocks/erie.us.txt  
  inflating: /content/dataset/Stocks/erii.us.txt  
  inflating: /content/dataset/Stocks/erj.us.txt  
  inflating: /content/dataset/Stocks/erm.us.txt  
  inflating: /content/dataset/Stocks/ern.us.txt  
  inflating: /content/dataset/Stocks/eros.us.txt  
  inflating: /content/dataset/Stocks/eryp.us.txt  
  inflating: /content/dataset/Stocks/es.us.txt  
  inflating: /content/dataset/Stocks/esba.us.txt  
  inflating: /content/dataset/Stocks/esbk.us.txt  
  inflating: /content/dataset/Stocks/esca.us.txt  
  inflating: /content/dataset/Stocks/esdi.us.txt  
  inflating: /content/dataset/Stocks/esdiw.us.txt  
  inflating: /content/dataset/Stocks/ese.us.txt  
  inflating: /content/dataset/Stocks/e

In [5]:
data = pd.read_csv('/content/dataset/Stocks/hpq.us.txt', sep=',')
data = data[['Date', 'Open', 'High', 'Low', 'Close']]
data['Date'] = pd.to_datetime(data['Date'], utc=False)
data = data.sort_values('Date')

mask = data['Date'] >= '1999-12-01'

data = data.loc[mask]
data = data.reset_index(drop=True)
data.head()

,Date,Open,High,Low,Close
0,1999-12-01,18.205,18.672,18.194,18.470
1,1999-12-02,19.033,19.404,18.754,19.093
2,1999-12-03,19.678,20.514,19.567,20.095
3,1999-12-06,20.489,20.635,19.475,20.034
4,1999-12-07,19.773,21.029,19.773,20.348


In [6]:
data['Mid Low-High'] = (data['Low'] + data['High']) / 2.0
data.head()

,Date,Open,High,Low,Close,Mid Low-High
0,1999-12-01,18.205,18.672,18.194,18.470,18.4330
1,1999-12-02,19.033,19.404,18.754,19.093,19.0790
2,1999-12-03,19.678,20.514,19.567,20.095,20.0405
3,1999-12-06,20.489,20.635,19.475,20.034,20.0550
4,1999-12-07,19.773,21.029,19.773,20.348,20.4010


In [7]:
x_col = data.columns[-1]
ts_col = data.columns[0]
x_col, ts_col

('Mid Low-High', 'Date')

In [8]:
s_data = data[[ts_col] + [x_col]]

scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler_data = scaler.fit_transform(s_data[x_col].values.reshape(-1, 1)).tolist()
scaler_data = pd.DataFrame(scaler_data, columns=[x_col])
s_data = pd.DataFrame(pd.concat([scaler_data, s_data[ts_col]], axis=1), columns=s_data.columns)

s_data.head()

,Date,Mid Low-High
0,1999-12-01,0.573977
1,1999-12-02,0.600121
2,1999-12-03,0.639035
3,1999-12-06,0.639622
4,1999-12-07,0.653625


In [9]:
fig = px.line(s_data, x=ts_col, y=x_col)
fig.show()

In [10]:
def split_sequence(data, ts_col, x_col, n_steps=3, n_outs=1):

    sequence = data[x_col].values
    ts = data.loc[n_steps:len(sequence)-n_outs, ts_col].values

    X, Y = [], []

    for i in range(len(sequence)):
        end_ix = i + n_steps

        if end_ix > (len(sequence) - n_outs):
            break

        seq_x = sequence[i: end_ix]


        if n_outs > 1:
            seq_y = np.array(sequence[end_ix: end_ix + n_outs])
        else:
            seq_y = np.array(sequence[end_ix])

        X.append(seq_x)
        Y.append(seq_y)


    X = np.array(X)
    Y = np.array(Y)

    if n_outs < 2:
        Y = np.expand_dims(Y, -1)

    columns = ['%s (t-%d)' % (x_col, i) for i in range(n_steps, 0, -1)]

    if n_outs > 1:
        columns = columns + ['%s (t)' % (x_col) if i == 0 else '%s (t+%d)' % (x_col, i) for i in range(0, n_outs)]
    else:
        columns = columns + ['%s (t)' % (x_col)]


    _data = pd.DataFrame(np.concatenate([X, Y], axis=1), columns=columns)
    _data[ts_col] = ts

    _data = _data[[ts_col] + columns]
    _data = _data.dropna()
    _data = _data.reset_index(drop=True)


    return _data

In [11]:
sequence_len = 20
n_outs = 1
new_data = split_sequence(s_data, ts_col, x_col, n_steps=sequence_len, n_outs=n_outs)
new_data.head()

,Date,Mid Low-High (t-20),Mid Low-High (t-19),Mid Low-High (t-18),Mid Low-High (t-17),Mid Low-High (t-16),Mid Low-High (t-15),Mid Low-High (t-14),Mid Low-High (t-13),Mid Low-High (t-12),...,Mid Low-High (t-9),Mid Low-High (t-8),Mid Low-High (t-7),Mid Low-High (t-6),Mid Low-High (t-5),Mid Low-High (t-4),Mid Low-High (t-3),Mid Low-High (t-2),Mid Low-High (t-1),Mid Low-High (t)
0,1999-12-30,0.573977,0.600121,0.639035,0.639622,0.653625,0.671331,0.683209,0.678272,0.659169,...,0.631628,0.655082,0.647089,0.652876,0.655264,0.682724,0.686265,0.686366,0.709273,0.726210
1,1999-12-31,0.600121,0.639035,0.639622,0.653625,0.671331,0.683209,0.678272,0.659169,0.634239,...,0.655082,0.647089,0.652876,0.655264,0.682724,0.686265,0.686366,0.709273,0.726210,0.715384
2,2000-01-03,0.639035,0.639622,0.653625,0.671331,0.683209,0.678272,0.659169,0.634239,0.611980,...,0.647089,0.652876,0.655264,0.682724,0.686265,0.686366,0.709273,0.726210,0.715384,0.726473
3,2000-01-04,0.639622,0.653625,0.671331,0.683209,0.678272,0.659169,0.634239,0.611980,0.631628,...,0.652876,0.655264,0.682724,0.686265,0.686366,0.709273,0.726210,0.715384,0.726473,0.692579
4,2000-01-05,0.653625,0.671331,0.683209,0.678272,0.659169,0.634239,0.611980,0.631628,0.655082,...,0.655264,0.682724,0.686265,0.686366,0.709273,0.726210,0.715384,0.726473,0.692579,0.659635


In [12]:
x_cols = new_data.columns[1:-n_outs]
y_cols = new_data.columns[-n_outs:]

print('Features: \n%s' % str(list(x_cols)))
print('Target: \n%s' % str(list(y_cols)))

Features: 
['Mid Low-High (t-20)', 'Mid Low-High (t-19)', 'Mid Low-High (t-18)', 'Mid Low-High (t-17)', 'Mid Low-High (t-16)', 'Mid Low-High (t-15)', 'Mid Low-High (t-14)', 'Mid Low-High (t-13)', 'Mid Low-High (t-12)', 'Mid Low-High (t-11)', 'Mid Low-High (t-10)', 'Mid Low-High (t-9)', 'Mid Low-High (t-8)', 'Mid Low-High (t-7)', 'Mid Low-High (t-6)', 'Mid Low-High (t-5)', 'Mid Low-High (t-4)', 'Mid Low-High (t-3)', 'Mid Low-High (t-2)', 'Mid Low-High (t-1)']
Target: 
['Mid Low-High (t)']


In [13]:
new_data[new_data.columns[1:]] = new_data[new_data.columns[1:]].astype('float32')


In [14]:
cut_point = 3200
cut_point_tv = cut_point - int(cut_point * 0.3)

train_data = new_data.iloc[:cut_point_tv]
valid_data = new_data.iloc[cut_point_tv:cut_point]
test_data = new_data.iloc[cut_point:]

x_train, y_train = train_data.loc[:, x_cols].values, train_data.loc[:, y_cols].values
x_valid, y_valid = valid_data.loc[:, x_cols].values, valid_data.loc[:, y_cols].values
x_test, y_test = test_data.loc[:, x_cols].values, test_data.loc[:, y_cols].values

x_train = np.expand_dims(x_train, -1)
x_valid = np.expand_dims(x_valid, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)
x_valid = x_valid.astype(np.float32)
x_test = x_test.astype(np.float32)

y_train = y_train.astype(np.float32)
y_valid = y_valid.astype(np.float32)
y_test = y_test.astype(np.float32)

print('Training %s %s' % (str(x_train.shape), str(y_train.shape)))
print('Validating %s %s' % (str(x_valid.shape), str(y_valid.shape)))
print('Testing %s %s' % (str(x_test.shape), str(y_test.shape)))

Training (2240, 20, 1) (2240, 1)
Validating (960, 20, 1) (960, 1)
Testing (1297, 20, 1) (1297, 1)


In [15]:
fig = make_subplots()

fig.add_trace(
    go.Scatter(x=train_data['Date'], y=train_data['Mid Low-High (t)'], name='Mid Low-High (t) Train'),
)

fig.add_trace(
    go.Scatter(x=valid_data['Date'], y=valid_data['Mid Low-High (t)'], name='Mid Low-High (t) Valid'),
)

fig.add_trace(
    go.Scatter(x=test_data['Date'], y=test_data['Mid Low-High (t)'], name='Mid Low-High (t) Test'),
)

fig.update_layout(
    title_text='Train/Valid/Test Split Version 1.0'
)

fig.show()

In [18]:
def timeseries_split(data, ts_col, valid_step=100, valid_range=5, test_size=0.2):
    _data = data.copy()
    _data[ts_col] = pd.to_datetime(_data[ts_col], utc=False)

    train_test_splitter = len(_data) - int(len(_data) * test_size)

    _train_data = _data.iloc[:train_test_splitter]
    test_data = _data.iloc[train_test_splitter:]

    train_rngs = [[i, i + valid_step] for i in list(range(valid_step, len(_train_data), valid_step))]

    valid_rngs = train_rngs[5:][::valid_range]
    valid_data = []
    for valid_rng in valid_rngs:
        train_rngs.pop(train_rngs.index(valid_rng))
        valid_data.append(_train_data.iloc[valid_rng[0]: valid_rng[1]])

    train_data = []
    for train_rng in train_rngs:
        train_data.append(_train_data.iloc[train_rng[0]: train_rng[1]])

    # Change is here: Use axis=0 as a keyword argument
    train_data = pd.concat(train_data, axis=0)
    valid_data = pd.concat(valid_data, axis=0)


    train_data = train_data.reset_index(drop=True)
    valid_data = valid_data.reset_index(drop=True)
    test_data = test_data.reset_index(drop=True)

    return train_data, valid_data, test_data

In [19]:
train_data, valid_data, test_data = timeseries_split(new_data, ts_col, valid_step=40, valid_range=3, test_size=0.25)

x_train, y_train = train_data.loc[:, x_cols].values, train_data.loc[:, y_cols].values
x_valid, y_valid = valid_data.loc[:, x_cols].values, valid_data.loc[:, y_cols].values
x_test, y_test = test_data.loc[:, x_cols].values, test_data.loc[:, y_cols].values

x_train = np.expand_dims(x_train, -1)
x_valid = np.expand_dims(x_valid, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)
x_valid = x_valid.astype(np.float32)
x_test = x_test.astype(np.float32)

y_train = y_train.astype(np.float32)
y_valid = y_valid.astype(np.float32)
y_test = y_test.astype(np.float32)

print('Training %s %s' % (str(x_train.shape), str(y_train.shape)))
print('Validating %s %s' % (str(x_valid.shape), str(y_valid.shape)))
print('Testing %s %s' % (str(x_test.shape), str(y_test.shape)))

Training (2280, 20, 1) (2280, 1)
Validating (1053, 20, 1) (1053, 1)
Testing (1124, 20, 1) (1124, 1)


In [20]:
fig = make_subplots()

fig.add_trace(
    go.Scatter(x=train_data['Date'], y=train_data['Mid Low-High (t)'], name='Mid Low-High (t) Train'),
)

fig.add_trace(
    go.Scatter(x=valid_data['Date'], y=valid_data['Mid Low-High (t)'], name='Mid Low-High (t) Valid'),
)

fig.add_trace(
    go.Scatter(x=test_data['Date'], y=test_data['Mid Low-High (t)'], name='Mid Low-High (t) Test'),
)

fig.update_layout(
    title_text='Train/Valid/Test Split Version 2.0'
)

fig.show()

In [ ]:
[n_timesteps, n_features], n_outputs = x_train.shape[1:], y_train.shape[1]
print('#%d Timesteps, #%d Features, #%d Target' % (n_timesteps, n_features, n_outputs))

In [25]:
def build_model(n_timesteps, n_features, n_outputs=1, rnn_units=None, dropout_rate=0.3, lr=1e-4):
    rnn_units = rnn_units if isinstance(rnn_units, list) else [100, 100]

    inputs = tf.keras.layers.Input(shape=[n_timesteps, n_features], name='inputs')

    x = inputs

    for units in rnn_units[:-1]:
        x = tf.keras.layers.LSTM(units, return_sequences=True)(x)
        x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

    x = tf.keras.layers.LSTM(rnn_units[-1], return_sequences=False)(x)
    x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

    outputs = tf.keras.layers.Dense(n_outputs, activation='linear')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Change 'lr' to 'learning_rate'
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss='mse')

    return model

In [26]:
[n_timesteps, n_features], n_outputs = x_train.shape[1:], y_train.shape[1]
print('#%d Timesteps, #%d Features, #%d Target' % (n_timesteps, n_features, n_outputs))

#20 Timesteps, #1 Features, #1 Target


In [27]:
model = build_model(n_timesteps, n_features, n_outputs=n_outputs, rnn_units=[200, 200, 100], dropout_rate=0.2, lr=1e-3)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)                  │ (None, 20, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 20, 200)             │         161,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 20, 200)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 20, 200)             │         320,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 20, 200)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 100)                 │         120,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 602,901 (2.30 MB)

 Trainable params: 602,901 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
r = model.fit(
    x_train, y_train,
    validation_data=(x_valid, y_valid),
    batch_size=128,
    verbose=1,
    epochs=20)


history_dict = r.history

# Move the plotting code here
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = list(range(1, len(loss) + 1))

fig = make_subplots()
fig.add_trace(
    go.Scatter(x=epochs, y=loss, name='Training Loss'),
)
fig.add_trace(
    go.Scatter(x=epochs, y=val_loss, name='Validation loss'),
)
fig.update_layout(
    title_text='Training/Validation Loss'
)
fig.show()

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 396ms/step - loss: 0.0013 - val_loss: 3.9271e-04
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 315ms/step - loss: 0.0013 - val_loss: 3.9444e-04
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 316ms/step - loss: 0.0011 - val_loss: 3.7272e-04
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 479ms/step - loss: 0.0013 - val_loss: 4.7501e-04
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 317ms/step - loss: 0.0012 - val_loss: 5.1688e-04
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 313ms/step - loss: 0.0014 - val_loss: 3.9075e-04
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 406ms/step - loss: 0.0012 - val_loss: 5.0353e-04
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 415ms/step - loss: 0.0012 - val_loss: 4.0584e-04
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 327ms/step - loss: 0.0010 - val_loss: 3.9482e-04
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 328ms/step - loss: 0.0011 - val_loss: 3.5269e-04
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 383ms/step - loss: 0.0012 - val_loss: 3.4550e-04
Ep

In [31]:
model.evaluate(x_test, y_test, verbose=1)

36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 6.3471e-04


0.0009555880096741021

In [32]:
def prediction_point_by_point(model, x):
    predicted = model.predict(x)
    return predicted

In [33]:
xx = x_test
yy = y_test
tt = test_data

if len(yy.shape) > 1:
    yy = scaler.inverse_transform(yy)
else:
    yy = scaler.inverse_transform(yy.reshape(-1, 1)).flatten()

In [34]:
pp = prediction_point_by_point(model, xx)

if len(pp.shape) > 1:
    pp = scaler.inverse_transform(pp)
else:
    pp = scaler.inverse_transform(pp.reshape(-1, 1)).flatten()

36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step


In [35]:
fig = make_subplots()

fig.add_trace(
    go.Scatter(x=tt['Date'], y=yy[:, 0], name='True Target (t)'),
)
fig.add_trace(
    go.Scatter(x=tt['Date'], y=pp[:, 0], name='Predicted (t)'),
)
fig.update_layout(
    title_text='Point-By-Point Prediction'
)

fig.show()

In [51]:
def prediction_by_date(model, dt, date_time_str, x_cols, y_cols, scaler, tc='Date', tcf='%Y-%m-%d', freq=7):
    # Convert 'date_time_str' to datetime for comparison
    # Use %d %b %Y to match the abbreviated month format
    # The original format string was too strict.
    # This updated format allows for variable whitespace.
    # Changed to format='%d %b %Y' to match the abbreviated month format
    date_time_obj = pd.to_datetime(date_time_str, format='%d %b %Y')

    cdt = dt[dt[tc] == date_time_obj].index
    cdt = list(cdt)

    if not len(cdt) > 0:
        print('There is no data with your specific datetime [%s]' % date_time_str)
        return False, None  # Return an empty DataFrame instead of None

    indices = [cdt[0] + i for i in range(freq)]
    cdt = dt.iloc[indices]

    ts = cdt[tc]
    tsf = ts.dt.strftime(tcf).values
    y = cdt[y_cols].values.flatten()
    x = cdt[x_cols].values[:1]
    p = []

    for i in range(freq):
        x_reshaped = x.reshape(1, x.shape[1], 1)
        pp = model.predict(x_reshaped)
        x = np.concatenate([x[:, 1:], pp], 1)
        p.append(pp[0][0])

    # Create DataFrame for predicted values
    p = np.array(p).reshape(-1, 1)
    p = scaler.inverse_transform(p)

    # Create the DataFrame
    df = pd.DataFrame({'Date': ts, 'P': p.flatten()})


    return True, df  # Return True and the DataFrame

In [52]:
trend_data = {}
dates = ['14 Nov 2013', '14 March 2014', '3 Sep 2014', '6 Nov 2014', '24 Feb 2015', '13 July 2015', '3 March 2016', '1 Feb 2017', '7 July 2017',]
for date in dates:
    print('Start from %s' % date)
    s, df = prediction_by_date(model, tt, date, x_cols=x_cols, y_cols=y_cols, freq=20, scaler=scaler)
    if s:
        trend_data[date] = df
    print()

Start from 14 Nov 2013
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

Start from 14 March 2014


ValueError: time data "14 March 2014" doesn't match format "%d %b %Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [53]:
fig = make_subplots()

fig.add_trace(
    go.Scatter(x=tt['Date'], y=yy[:, 0], name='True Target (t)', line=dict(width=2, color='#512b58')),
)
fig.add_trace(
    go.Scatter(x=tt['Date'], y=pp[:, 0], name='Predicted (t)', line=dict(width=2, color="#40bad5")),
)

for k, df in trend_data.items():
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['P'], name='Trend [%s]' % k, mode='lines', line=dict(width=4, color='#d8345f')),
    )

fig.update_layout(
    title_text='Trend Prediction'
)

fig.show()